In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
ticker = "TSLA"

In [3]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql import Window
import pandas as pd
import numpy as np
import pyarrow
import yfinance as yf

In [4]:
import yfinance as yf
tsla = yf.Ticker(ticker)
tsla.info

{'zip': '94304',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 48016,
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, Netherlands, Norway, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers sedans and sport utility vehicles. It also provides electric powertrain components and systems; and services for electric vehicles through its company-owned service locations, and Tesla mobile service technicians, as well as sells used vehicles. This segment markets and sells its products through a network of company-owned stores and galleries, as well as through its own Website. The Energy Generation and Storage segment offers energy storage products, such as rechargeable lithium-ion battery systems for use in homes, industrial, commercial facilities, and utility grids; and desig

In [5]:
df = tsla.history(period="30d")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-04-03,509.50,515.49,468.39,480.01,22562100,0,0
2020-04-06,511.20,521.00,497.96,516.24,14901800,0,0
2020-04-07,545.00,565.00,532.34,545.45,17919800,0,0
2020-04-08,554.20,557.21,533.33,548.84,12656000,0,0
2020-04-09,562.09,575.18,557.11,573.00,13650000,0,0
2020-04-13,590.16,652.00,580.53,650.95,22475400,0,0
2020-04-14,698.97,741.88,692.43,709.89,30576500,0,0
2020-04-15,742.00,753.13,710.00,729.83,23577000,0,0
2020-04-16,716.94,759.45,706.72,745.21,20657900,0,0


In [6]:
sparkdf = spark.createDataFrame(df)

In [7]:
#spark.conf.set("spark.sql.execution.arrow.enabled","true")
sparkdf = sqlContext.createDataFrame(df.reset_index(drop=False))
sparkdf

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: bigint, Dividends: bigint, Stock Splits: bigint]

In [8]:
sparkdf

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: bigint, Dividends: bigint, Stock Splits: bigint]

In [9]:
sparkdf.show()

+-------------------+------+------+------+------+--------+---------+------------+
|               Date|  Open|  High|   Low| Close|  Volume|Dividends|Stock Splits|
+-------------------+------+------+------+------+--------+---------+------------+
|2020-04-03 00:00:00| 509.5|515.49|468.39|480.01|22562100|        0|           0|
|2020-04-06 00:00:00| 511.2| 521.0|497.96|516.24|14901800|        0|           0|
|2020-04-07 00:00:00| 545.0| 565.0|532.34|545.45|17919800|        0|           0|
|2020-04-08 00:00:00| 554.2|557.21|533.33|548.84|12656000|        0|           0|
|2020-04-09 00:00:00|562.09|575.18|557.11| 573.0|13650000|        0|           0|
|2020-04-13 00:00:00|590.16| 652.0|580.53|650.95|22475400|        0|           0|
|2020-04-14 00:00:00|698.97|741.88|692.43|709.89|30576500|        0|           0|
|2020-04-15 00:00:00| 742.0|753.13| 710.0|729.83|23577000|        0|           0|
|2020-04-16 00:00:00|716.94|759.45|706.72|745.21|20657900|        0|           0|
|2020-04-17 00:0

In [10]:
#df_pandas = df_pandas.drop(['Dividends', 'Stock Splits'], axis=1)
sparkdf = sparkdf.drop('Dividends','Stock Splits')

In [11]:
sparkdf.show()

+-------------------+------+------+------+------+--------+
|               Date|  Open|  High|   Low| Close|  Volume|
+-------------------+------+------+------+------+--------+
|2020-04-03 00:00:00| 509.5|515.49|468.39|480.01|22562100|
|2020-04-06 00:00:00| 511.2| 521.0|497.96|516.24|14901800|
|2020-04-07 00:00:00| 545.0| 565.0|532.34|545.45|17919800|
|2020-04-08 00:00:00| 554.2|557.21|533.33|548.84|12656000|
|2020-04-09 00:00:00|562.09|575.18|557.11| 573.0|13650000|
|2020-04-13 00:00:00|590.16| 652.0|580.53|650.95|22475400|
|2020-04-14 00:00:00|698.97|741.88|692.43|709.89|30576500|
|2020-04-15 00:00:00| 742.0|753.13| 710.0|729.83|23577000|
|2020-04-16 00:00:00|716.94|759.45|706.72|745.21|20657900|
|2020-04-17 00:00:00|772.28|774.95|747.66|753.89|13128200|
|2020-04-20 00:00:00| 732.7|765.57|712.21|746.36|14746600|
|2020-04-21 00:00:00|730.12|753.33|673.79|686.72|20209100|
|2020-04-22 00:00:00|703.98| 734.0|688.71|732.11|14224800|
|2020-04-23 00:00:00| 727.6| 734.0|703.13|705.63|1323670

In [12]:
df = sparkdf.toPandas()
df

,Date,Open,High,Low,Close,Volume
0,2020-04-03,509.50,515.49,468.39,480.01,22562100
1,2020-04-06,511.20,521.00,497.96,516.24,14901800
2,2020-04-07,545.00,565.00,532.34,545.45,17919800
3,2020-04-08,554.20,557.21,533.33,548.84,12656000
4,2020-04-09,562.09,575.18,557.11,573.00,13650000
5,2020-04-13,590.16,652.00,580.53,650.95,22475400
6,2020-04-14,698.97,741.88,692.43,709.89,30576500
7,2020-04-15,742.00,753.13,710.00,729.83,23577000
8,2020-04-16,716.94,759.45,706.72,745.21,20657900
9,2020-04-17,772.28,774.95,747.66,753.89,13128200


In [28]:
df['Dollar_Change'] = round(df.Close.diff(), 2)
df['Percent_Change'] = df.Close.pct_change().round(4)
df

,Date,Open,High,Low,Close,Volume,Dollar_Change,Percent_Change
0,2020-04-03,509.50,515.49,468.39,480.01,22562100,NaN,NaN
1,2020-04-06,511.20,521.00,497.96,516.24,14901800,36.23,0.0755
2,2020-04-07,545.00,565.00,532.34,545.45,17919800,29.21,0.0566
3,2020-04-08,554.20,557.21,533.33,548.84,12656000,3.39,0.0062
4,2020-04-09,562.09,575.18,557.11,573.00,13650000,24.16,0.0440
5,2020-04-13,590.16,652.00,580.53,650.95,22475400,77.95,0.1360
6,2020-04-14,698.97,741.88,692.43,709.89,30576500,58.94,0.0905
7,2020-04-15,742.00,753.13,710.00,729.83,23577000,19.94,0.0281
8,2020-04-16,716.94,759.45,706.72,745.21,20657900,15.38,0.0211
9,2020-04-17,772.28,774.95,747.66,753.89,13128200,8.68,0.0116


In [25]:
# df_fin = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dollar_Change', 'Percent_Change']]
# df_fin
#from pyspark.sql.functions import lit
#sparkdf = sparkdf.withColumn("preclose", lit(0))
#sparkdf.show()

In [27]:
df.to_csv('yfinance.csv')